In [1]:
import os
import numpy as np
import trimesh
import torch
from models.networks.smpl import SMPL
from utils.util import load_obj, load_pickle_file, write_pickle_file

In [2]:
data_root = 'data/Multi-Garment_dataset'
garment_classes = ['Pants', 'ShortPants', 'ShirtNoCoat', 'TShirtNoCoat', 'LongCoat']

In [3]:
people_IDs_list = os.listdir(data_root)
print(len(people_IDs_list))

96


In [4]:
smpl = SMPL(pkl_path='assets/smpl_model.pkl', isHres=True).cuda()

In [5]:
def get_shape_pose_cam_v_personal(smpl, people_ID, device='cuda:0'):

    smpl_registration_pkl = load_pickle_file(os.path.join(data_root, people_ID, 'registration.pkl'))
        
    shape = torch.from_numpy(smpl_registration_pkl['betas']).float().to(device)
    pose = torch.from_numpy(smpl_registration_pkl['pose']).float().to(device)
    
    pose_T = torch.zeros(pose.shape).float().to(device)
    
    verts_T = smpl(shape[None], pose_T[None])[0]
    
    cam = torch.zeros(3).float().to(device)
    cam[0] = (1 - torch.rand(1) * 0.2) / verts_T[:, 0:1].abs().max()
    cam[1] = - (verts_T[:, 0].min() + verts_T[:, 0].max()) / 2
    cam[2] = - (verts_T[:, 1].min() + verts_T[:, 1].max()) / 2
    
    v_personal = torch.zeros(verts_T.shape).to(device)
    print("---"+people_ID+"---")
    for garment_type in garment_classes:
        garment_obj_path = os.path.join(data_root, people_ID, garment_type+'.obj')
        vert_inds = torch.from_numpy(vert_indices[garment_type])
        if os.path.isfile(garment_obj_path):
            garment_obj = load_obj(garment_obj_path)
            garment_v = torch.from_numpy(garment_obj['vertices']).float().to(device)
            v_personal[vert_inds] = garment_v - verts_T[vert_inds]
            print(garment_type)
    return shape, pose, cam, v_personal

In [6]:
def get_hres(v, f):
    """
    Get an upsampled version of the mesh.
    OUTPUT:
        - nv: new vertices
        - nf: faces of the upsampled
        - mapping: mapping from low res to high res
    """
    from opendr.topology import loop_subdivider
    (mapping, nf) = loop_subdivider(v, f)
    nv = mapping.dot(v.ravel()).reshape(-1, 3)
    return (nv, nf, mapping)

def get_vt_ft():
    vt, ft = load_pickle_file('assets/smpl_vt_ft.pkl')
    return vt, ft

def get_vt_ft_hres():
    vt, ft = get_vt_ft()
    vt, ft, _ = get_hres(np.hstack((vt, np.ones((vt.shape[0], 1)))), ft)
    return vt[:, :2], ft

In [7]:
vt, ft = get_vt_ft()
print(vt.shape, ft.shape)

(7576, 2) (13776, 3)


In [8]:
vt_hres, ft_hres = get_vt_ft_hres()
print(vt_hres.shape, ft_hres.shape)

(28920, 2) (55104, 3)


In [13]:
write_pickle_file('assets/smpl_vt_ft_hres.pkl', (vt_hres, ft_hres))

In [ ]:
vert_indices, fts = load_pickle_file('assets/garment_fts.pkl')
fts['naked'] = ft_hres
for key in vert_indices:
    print(key, vert_indices[key].shape)
print('------------')
for key in fts:
    print(key, fts[key].shape)

In [ ]:
for people_ID in people_IDs_list:
    shape, pose, cam, v_personal = get_shape_pose_cam_v_personal(smpl, people_ID)
    smpl_registered_pkl = {'betas': shape.cpu().numpy(), 'pose': pose.cpu().numpy(), 'camera': cam.cpu().numpy(), 'v_personal': v_personal.cpu().numpy()}
    write_pickle_file(os.path.join(data_root, people_ID, 'smpl_registered.pkl'), smpl_registered_pkl)

In [ ]:
vis_ID = 0
people_ID = people_IDs_list[vis_ID]
smpl_registered_pkl = load_pickle_file(os.path.join(data_root, people_ID, 'smpl_registered.pkl'))
shape = torch.from_numpy(smpl_registered_pkl['betas']).float().cuda()
pose = torch.from_numpy(smpl_registered_pkl['pose']).float().cuda()
v_personal = torch.from_numpy(smpl_registered_pkl['v_personal']).float().cuda()

In [ ]:
verts = smpl(shape[None], pose[None])[0]

In [ ]:
mesh = trimesh.Trimesh(vertices=verts.cpu(), faces=smpl.faces_hres.cpu(), process=False)
mesh.show()

In [ ]:
verts_personal = smpl(shape[None], pose[None], v_personal[None])[0]

In [ ]:
mesh = trimesh.Trimesh(vertices=verts_personal.cpu(), faces=smpl.faces_hres.cpu(), process=False)
mesh.show()